In [3]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving conflict-data-for-indonesia-2 - conflict-data-for-indonesia-2.csv to conflict-data-for-indonesia-2 - conflict-data-for-indonesia-2.csv


In [5]:
df = pd.read_csv('C:\Users\radis\OneDrive\Desktop\SEM5 Mat\DATSCI\DatSciFinalSem5\Data\data2.csv')

df.drop(index=df.index[0],axis=0,inplace=True)

df.info()
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data2.csv'

## Finding and fixing missing data

In [ ]:
sum(df.apply(lambda x:sum(x.isnull().values),axis=1)>0)

1725

1725 rows with missing values means each row has at least 1 missing value. We must first find which column that has the most missing values and remove them, as well as using fillna to clean the data for processing

In [ ]:
print(df.isnull().sum())

id                      0
year                    0
active_year             0
start_year              0
end_year                0
type_of_violence        0
conflict_new_id         0
conflict_name           0
dyad_new_id             0
dyad_name               0
side_a_new_id           0
gwnoa                 247
side_a                  0
side_b_new_id           0
gwnob                1725
side_b                  0
number_of_sources       0
source_article          9
source_office        1208
source_date          1208
source_headline      1208
source_original       498
where_prec              0
where_coordinates       0
adm_1                   6
adm_2                 251
latitude                0
longitude               0
geom_wkt                0
priogrid_gid            0
country                 0
country_id              0
iso3                    0
region                  0
event_clarity           0
date_prec               0
date_start              0
date_end                0
deaths_a    

The columns with most missing values are:


*   GWNoA (Gledistch and Ward country codes of Side A)
*   GWNoB (Gledistch and Ward country codes of Side B)
*   Source_Office
*   Source_Date
*   Source_Headline
*   Source_Original
*   Source_Article
*   Adm_1
*   Adm_2







In [ ]:
dropcolumns = ['gwnoa','gwnob','source_office','source_date','source_headline','source_original','adm_1','adm_2','source_article']
for c in dropcolumns:
  df.drop(c,axis=1,inplace=True)

print(df.isnull().sum())

id                   0
year                 0
active_year          0
start_year           0
end_year             0
type_of_violence     0
conflict_new_id      0
conflict_name        0
dyad_new_id          0
dyad_name            0
side_a_new_id        0
side_a               0
side_b_new_id        0
side_b               0
number_of_sources    0
where_prec           0
where_coordinates    0
latitude             0
longitude            0
geom_wkt             0
priogrid_gid         0
country              0
country_id           0
iso3                 0
region               0
event_clarity        0
date_prec            0
date_start           0
date_end             0
deaths_a             0
deaths_b             0
deaths_civilians     0
deaths_unknown       0
low                  0
best                 0
high                 0
dtype: int64


Now that there are no more missing values, we filter out which data we are going to use for processing.

After lengthy discussions, we decided that the columns that we are going to use are:


In [ ]:
uniquecolumns = []
for columns in df:
  uniquecolumns.append(columns)

for c in uniquecolumns:
  print("\n",c,": ")
  uniquevals = df[c].unique()
  print(np.sort(a=uniquevals))




 id : 
[ 68299.  68300.  68319. ... 273583. 274960. 276379.]

 year : 
['1989' '1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998'
 '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008'
 '2011' '2012' '2013' '2014' '2015' '2017' '2018']

 active_year : 
[0. 1.]

 start_year : 
['1989' '1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998'
 '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008'
 '2011' '2012' '2013' '2014' '2015' '2017' '2018']

 end_year : 
['1989' '1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998'
 '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008'
 '2011' '2012' '2013' '2014' '2015' '2017' '2018']

 type_of_violence : 
[1. 2. 3.]

 conflict_new_id : 
[ 291.  330.  366.  493.  524.  539.  630. 4755. 4903. 4929.]

 conflict_name : 
['Christians (Indonesia) - Muslims (Indonesia)' 'Dayak - Madurese'
 'Dayak, Malay (Indonesia) - Madurese' 'GAM - Civilians'
 'Government of Indonesia - Civilia